# 0. Initialisation

In [3]:
import pandas as pd
import numpy as np 
from helpers2 import S3Connection

In [4]:
s3 = S3Connection(bucket_name="clichere/diffusion")

Connection successful


# 1. Ouverture et Nettoyage des fichiers DPE

## 1.1. DPE après juillet 2021

In [26]:
path_logements_existants = "DPE/DPE_ADEME/dpe-v2-logements-existants.csv"
DPE_logements_existants = s3.read_csv_from_s3(path_logements_existants)

In [27]:
print(DPE_logements_existants.columns)

Index(['Date_réception_DPE', 'Date_établissement_DPE', 'Modèle_DPE',
       'Date_fin_validité_DPE', 'Version_DPE', 'Méthode_application_DPE',
       'Etiquette_DPE', 'Etiquette_GES', 'Année_construction', 'Type_bâtiment',
       'Période_construction', 'Surface_habitable_logement', 'Adresse_brute',
       'Nom__commune_(BAN)', 'Code_INSEE_(BAN)', 'N°_voie_(BAN)',
       'Identifiant__BAN', 'Adresse_(BAN)', 'Code_postal_(BAN)', 'Score_BAN',
       'Nom__rue_(BAN)', 'Coordonnée_cartographique_X_(BAN)',
       'Coordonnée_cartographique_Y_(BAN)', 'Code_postal_(brut)',
       'N°_étage_appartement', 'Nom_résidence', 'Cage_d'escalier',
       'Complément_d'adresse_logement', 'Statut_géocodage',
       'Nom__commune_(Brut)', 'N°_département_(BAN)', 'N°_région_(BAN)',
       'Complément_d'adresse_bâtiment'],
      dtype='object')


In [21]:
print(DPE_logements_existants["Code_INSEE_(BAN)"].value_counts(dropna=False))

Code_INSEE_(BAN)
31555        51965
44109        37929
33063        32239
06088        28698
51454        27173
             ...  
AVION            1
39527            1
old 80621        1
old 49069        1
old 15195        1
Name: count, Length: 34493, dtype: int64


In [32]:
DPE_logements_existants_clean = DPE_logements_existants[[ 'Date_établissement_DPE',
       'Etiquette_DPE', 'Type_bâtiment',
       'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)' ,'N°_étage_appartement',
       'N°_département_(BAN)', 'N°_région_(BAN)','Code_INSEE_(BAN)', 'Version_DPE']]

In [33]:
DPE_logements_existants_clean.sample(10)

,Date_établissement_DPE,Etiquette_DPE,Type_bâtiment,Période_construction,Surface_habitable_logement,Adresse_(BAN),N°_étage_appartement,N°_département_(BAN),N°_région_(BAN),Code_INSEE_(BAN),Version_DPE
2776528,2022-04-21,C,appartement,2006-2012,42.0,Chemin de Villechien 49124 Saint-Barthélemy-d'...,0.0,49,52.0,49267,2.1
1819893,2023-03-08,D,appartement,avant 1948,37.8,65 Route du Général de Gaulle 67300 Schiltigheim,0.0,67,44.0,67447,2.2
4066806,2022-11-22,C,appartement,1948-1974,63.0,NaN,NaN,NaN,NaN,NaN,2.2
2265407,2023-05-10,D,appartement,1948-1974,45.0,8 Rue Marcel Cachin 91270 Vigneux-sur-Seine,0.0,91,11.0,91657,2.2
4085245,2022-06-18,C,appartement,1948-1974,83.0,41 rue Dumont d'Urville 51100 Reims,NaN,51,44.0,51454,2.1
3313492,2022-09-05,C,appartement,1975-1977,66.0,3 Allee des Iris 56000 Vannes,NaN,56,53.0,56260,2.1
253050,2023-02-23,E,immeuble,avant 1948,NaN,15 Rue de la Croix de Jubilé 79140 Bretignolles,0.0,79,75.0,79050,2.2
1028677,2022-06-16,E,maison,1989-2000,34.7,NaN,0.0,NaN,NaN,NaN,2.1
1913309,2021-09-30,G,appartement,avant 1948,24.2,2 Place Magenta 06000 Nice,NaN,06,93.0,06088,1.0
134327,2021-12-21,D,appartement,1948-1974,47.9,Rue de la Mayenne 44800 Saint-Herblain,NaN,44,52.0,44162,2.0


In [34]:
print(DPE_logements_existants_clean.shape)

(4537525, 11)


In [35]:
doublons_DPE = DPE_logements_existants_clean.duplicated().sum()
total_obs_DPE = len(DPE_logements_existants_clean)
ratio_doublons_DPE = doublons_DPE/total_obs_DPE
print(ratio_doublons_DPE)

0.18282081090462313


Environ 18,3% des observations sont des doublons, enlevons les

In [36]:
DPE_logements_existants_clean = DPE_logements_existants_clean.drop_duplicates()

## 1.2. DPE avant juillet 2021

In [128]:
outpath1 = 'DPE/DPE_après_2021.parquet'

s3.from_pandas_to_parquet_store_in_s3(DPE_logements, outpath1)

Fichier Parquet écrit avec succès dans DPE/DPE_après_2021.parquet


# 2. Ouverture et nettoyage des fichiers valeurs foncières

## 2.1. Téléchargement et fusion des fichiers csv

In [3]:
path_vf_2024 = "valeursfoncieres/vf_2024.csv"
vf_2024 = s3.read_csv_from_s3(path_vf_2024)

path_vf_2023 = "valeursfoncieres/vf_2023.csv"
vf_2023 = s3.read_csv_from_s3(path_vf_2023)

path_vf_2022 = "valeursfoncieres/vf_2022.csv"
vf_2022 = s3.read_csv_from_s3(path_vf_2022)

path_vf_2021 = "valeursfoncieres/vf_2021.csv"
vf_2021 = s3.read_csv_from_s3(path_vf_2021)

path_vf_2020 = "valeursfoncieres/vf_2020.csv"
vf_2020 = s3.read_csv_from_s3(path_vf_2020)

path_vf_2019 = "valeursfoncieres/vf_2019.csv"
vf_2019 = s3.read_csv_from_s3(path_vf_2019)

In [6]:
print(vf_2019["valeur_fonciere"].describe())
print(vf_2020["valeur_fonciere"].describe())
print(vf_2021["valeur_fonciere"].describe())
print(vf_2022["valeur_fonciere"].describe())
print(vf_2023["valeur_fonciere"].describe())
print(vf_2024["valeur_fonciere"].describe())
print(vf_2019.columns)

count    1.945453e+06
mean     1.006466e+06
std      8.090271e+06
min      1.000000e-02
25%      6.450000e+04
50%      1.550000e+05
75%      2.821490e+05
max      8.450000e+08
Name: valeur_fonciere, dtype: float64
count    3.471436e+06
mean     8.727544e+05
std      7.780260e+06
min      1.000000e-02
25%      6.655000e+04
50%      1.550000e+05
75%      2.800000e+05
max      4.900000e+08
Name: valeur_fonciere, dtype: float64
count    4.629636e+06
mean     2.006912e+06
std      2.036543e+07
min      1.000000e-01
25%      7.250000e+04
50%      1.660000e+05
75%      3.060000e+05
max      1.415000e+10
Name: valeur_fonciere, dtype: float64
count    4.635517e+06
mean     2.801775e+06
std      2.677626e+07
min      1.000000e-02
25%      7.600000e+04
50%      1.750000e+05
75%      3.200000e+05
max      1.003401e+09
Name: valeur_fonciere, dtype: float64
count    3.765454e+06
mean     6.174104e+05
std      5.111104e+06
min      1.000000e-01
25%      6.156900e+04
50%      1.600000e+05
75%      3.0

In [7]:
vf_new = pd.concat([vf_2024, vf_2023, vf_2022, vf_2021, vf_2020, vf_2019], ignore_index=True)


In [8]:
vf_new.sample(20)
print(vf_new.shape)
print(vf_new.columns)
print(vf_new["date_mutation"].value_counts)
doublons_vf_2024 = vf_new.duplicated()
print(f"nombre de doublons {doublons_vf_2024.sum()}")

(20195667, 40)
Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')
<bound method IndexOpsMixin.value_counts of 0           2024-01-02
1           2024-01-03
2           2024-01

In [9]:
vf_new.sample(10)

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
14048370,2021-1452065,2021-06-18,1,Vente,195000.0,223.0,NaN,AV DU HUIT MAI,0480,84500.0,...,Dépendance,NaN,0.0,S,sols,NaN,NaN,647.0,4.743620,44.283019
17068243,2020-956500,2020-10-01,1,Vente,4550000.0,7.0,NaN,RUE STE-CATHERINE,6610,69001.0,...,Appartement,58.0,3.0,S,sols,NaN,NaN,393.0,4.832843,45.768326
6462839,2022-369088,2022-05-31,1,Vente,974401.0,2710.0,NaN,CITE LAFAYETTE,1888,27000.0,...,Maison,135.0,1.0,AG,terrains d'agrément,NaN,NaN,1037.0,NaN,NaN
7090928,2022-609146,2022-03-28,1,Vente,265000.0,NaN,NaN,AV DU VERT GALANT,0192,35340.0,...,NaN,NaN,NaN,S,sols,NaN,NaN,1488.0,-1.509915,48.206710
15199416,2020-199194,2020-04-30,2,Vente,2485553.0,NaN,NaN,LE GRAND SETOUBE,B086,16120.0,...,NaN,NaN,NaN,VI,vignes,NaN,NaN,460.0,-0.103587,45.636849
863009,2024-296819,2024-04-22,1,Vente,100000.0,15.0,NaN,RUE PAUL DEROULEDE,0977,55100.0,...,Dépendance,NaN,0.0,S,sols,NaN,NaN,290.0,5.399196,49.162494
10193979,2021-54371,2021-01-29,1,Vente,425000.0,2112.0,R,RTE DE CASTELLAR,0024,6500.0,...,Maison,156.0,5.0,S,sols,NaN,NaN,1535.0,7.501570,43.795316
2362712,2023-268734,2023-04-03,1,Vente,92000.0,NaN,NaN,POMMIER,B057,24130.0,...,NaN,NaN,NaN,P,prés,NaN,NaN,1163.0,0.331378,44.929240
6356281,2022-332779,2022-10-11,1,Vente,227000.0,11.0,NaN,RUE GRAND RUE,2120,24100.0,...,Appartement,40.0,2.0,S,sols,NaN,NaN,133.0,0.482797,44.851879
15833781,2020-443363,2020-06-17,1,Vente,10000.0,NaN,NaN,ARREGOT EST,B006,33114.0,...,NaN,NaN,NaN,L,landes,NaN,NaN,14545.0,-0.703470,44.607162


In [26]:
print(vf_new["lot1_surface_carrez"].value_counts(dropna=False))
print(vf_new["lot2_surface_carrez"].value_counts(dropna=False))
print(vf_new["surface_reelle_bati"].value_counts(dropna=False))

lot1_surface_carrez
NaN       18442001
12.50         6410
12.00         3048
15.00         2285
10.00         1799
            ...   
390.13           1
278.46           1
1.17             1
169.99           1
203.22           1
Name: count, Length: 28348, dtype: int64
lot2_surface_carrez
NaN       19651067
65.00          445
60.00          436
70.00          417
68.00          403
            ...   
348.42           1
318.51           1
7.48             1
178.45           1
185.47           1
Name: count, Length: 17798, dtype: int64
surface_reelle_bati
NaN         13019863
80.0          141525
60.0          134169
70.0          126882
90.0          124531
              ...   
39667.0            1
78354.0            1
27057.0            1
107476.0           1
15318.0            1
Name: count, Length: 9346, dtype: int64


In [11]:
vf_new_clean = vf_new[['date_mutation', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'code_postal', 'code_commune',
       'nom_commune', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'surface_terrain', 'longitude', 'latitude']]

In [14]:
vf_new_clean['Adresse'] = vf_new_clean['adresse_numero'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf_new_clean['adresse_nom_voie'].fillna('').astype(str) + " " + \
                vf_new_clean['code_postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf_new_clean['nom_commune'].fillna('').astype(str)

vf_new_clean['Adresse'] = vf_new_clean['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides

/tmp/ipykernel_1880/1243246230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vf_new_clean['Adresse'] = vf_new_clean['adresse_numero'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
/tmp/ipykernel_1880/1243246230.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vf_new_clean['Adresse'] = vf_new_clean['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides


In [18]:
vf_new_clean.sample(10)

,date_mutation,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,code_postal,code_commune,nom_commune,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,Adresse
1308523,2024-03-27,Vente,263000.00,55.0,NaN,TRA FONT DE FILLOL,83140.0,83129,Six-Fours-les-Plages,Appartement,82.0,5.0,NaN,5.827281,43.097104,55 TRA FONT DE FILLOL 83140 Six-Fours-les-Plages
16930224,2020-02-05,Vente,77000.00,23.0,NaN,RUE SAINT GENES,63000.0,63113,Clermont-Ferrand,Dépendance,NaN,0.0,NaN,3.085754,45.776521,23 RUE SAINT GENES 63000 Clermont-Ferrand
10856620,2021-09-06,Vente,5029.20,NaN,NaN,CHAMPS PRUNIERS,21410.0,21578,Saint-Victor-sur-Ouche,NaN,NaN,NaN,920.0,4.735094,47.227558,CHAMPS PRUNIERS 21410 Saint-Victor-sur-Ouche
5776458,2022-12-16,Vente,302630.00,NaN,NaN,LE VILLAGE,11410.0,11359,Saint-Michel-de-Lanès,NaN,NaN,NaN,120.0,1.758577,43.324529,LE VILLAGE 11410 Saint-Michel-de-Lanès
9032064,2022-06-27,Vente,180000.00,1.0,NaN,RUE DU BELVEDERE,78750.0,78367,Mareil-Marly,Appartement,50.0,2.0,NaN,2.076576,48.882098,1 RUE DU BELVEDERE 78750 Mareil-Marly
6181542,2022-05-20,Vente,555000.00,7.0,B,RUE DU BAILLY,21000.0,21231,Dijon,Local industriel. commercial ou assimilé,165.0,0.0,1463.0,5.074335,47.350351,7 RUE DU BAILLY 21000 Dijon
7585874,2022-10-06,Vente,148000.00,147.0,NaN,ALL DES ROSIERS,45260.0,45230,Noyers,Maison,132.0,5.0,1354.0,2.568287,47.904617,147 ALL DES ROSIERS 45260 Noyers
9536518,2022-12-12,Vente,175000.00,3.0,NaN,RUE DE VERDUN,88110.0,88372,Raon-l'Étape,Dépendance,NaN,0.0,750.0,6.835251,48.401153,3 RUE DE VERDUN 88110 Raon-l'Étape
5136340,2023-10-26,Vente,68438.75,5.0,NaN,ALL JOAQUIM DU BELLAY,93390.0,93014,Clichy-sous-Bois,Appartement,66.0,4.0,NaN,2.543826,48.905517,5 ALL JOAQUIM DU BELLAY 93390 Clichy-sous-Bois
435390,2024-02-21,Vente,170000.00,NaN,NaN,LES LAUZIERES SUD,30900.0,30189,Nîmes,NaN,NaN,NaN,91573.0,4.261858,43.856613,LES LAUZIERES SUD 30900 Nîmes


In [20]:
vf_pre_final = vf_new_clean[['date_mutation', 'nature_mutation',
       'valeur_fonciere', 'Adresse', 'code_commune',
       'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'surface_terrain']]

In [ ]:
doublons_vf_new = vf_pre_final.duplicated()

print(f"nombre de doublons : {doublons_vf_new.sum()}")

nombre de doublons : 2604019


In [21]:
vf_final = vf_pre_final.drop_duplicates()

In [22]:
vf_final.sample(20)

,date_mutation,nature_mutation,valeur_fonciere,Adresse,code_commune,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain
9598046,2022-04-27,Vente,400000.0,15 AV ERIC MORLET 91320 Wissous,91689,Maison,75.0,2.0,275.0
16973894,2020-05-14,Vente,11500.0,BIOUS 64330 Portet,64455,NaN,NaN,NaN,4315.0
8073674,2022-05-13,Vente,125000.0,25 RUE DE WAILLY 59200 Tourcoing,59599,Appartement,83.0,4.0,NaN
15782186,2020-11-03,Vente,127200.0,A LA RIBERE OUEST 32000 Auch,32013,NaN,NaN,NaN,8180.0
11709066,2021-09-06,Vente,189500.0,3 RUE DES SAULES 35510 Cesson-Sévigné,35051,Dépendance,NaN,0.0,NaN
8890704,2022-12-16,Vente,169000.0,1 RUE MESSYRE D'ANDLAU 76130 Mont-Saint-Aignan,76451,Dépendance,NaN,0.0,NaN
9140485,2022-10-27,Vente,771950.5,2235 CHEM DES MOLLIERES 80120 Fort-Mahon-Plage,80333,Maison,87.0,4.0,635.0
9552442,2022-06-04,Vente,146000.0,CROIX JAPPEE 89520 Thury,89416,NaN,NaN,NaN,2925.0
2840437,2023-11-07,Vente,237650.0,20 AV DE LA GARE 33870 Vayres,33539,Dépendance,NaN,0.0,864.0
5069815,2023-12-04,Vente,1300000.0,38 RUE PARMENTIER 92200 Neuilly-sur-Seine,92051,Dépendance,NaN,0.0,NaN


## 2.2. Téléchargement et fusion des fichiers txt

In [3]:
dtype_spec = {"Code commune": str}

path_valeursfoncieres_2022 = "Valeursfoncières/valeursfoncieres-2022.txt"
vf2022 = s3.read_txt_from_s3(path_valeursfoncieres_2022, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2021 = "Valeursfoncières/valeursfoncieres-2021.txt"
vf2021 = s3.read_txt_from_s3(path_valeursfoncieres_2021, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2020 = "Valeursfoncières/valeursfoncieres-2020.txt"
vf2020 = s3.read_txt_from_s3(path_valeursfoncieres_2020, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2019 = "Valeursfoncières/valeursfoncieres-2019.txt"
vf2019 = s3.read_txt_from_s3(path_valeursfoncieres_2019, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2018 = "Valeursfoncières/valeursfoncieres-2018.txt"
vf2018 = s3.read_txt_from_s3(path_valeursfoncieres_2018, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2017 = "Valeursfoncières/valeursfoncieres-2017.txt"
vf2017 = s3.read_txt_from_s3(path_valeursfoncieres_2017, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2016 = "Valeursfoncières/valeursfoncieres-2016.txt"
vf2016 = s3.read_txt_from_s3(path_valeursfoncieres_2016, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2015 = "Valeursfoncières/valeursfoncieres-2015.txt"
vf2015 = s3.read_txt_from_s3(path_valeursfoncieres_2015, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2014 = "Valeursfoncières/valeursfoncieres-2014.txt"
vf2014 = s3.read_txt_from_s3(path_valeursfoncieres_2014, sep="|", dtype_spec=dtype_spec)


In [4]:
print(vf2014.shape)
print(vf2015.shape)
print(vf2016.shape)
print(vf2017.shape)
print(vf2018.shape)
print(vf2019.shape)
print(vf2020.shape)
print(vf2021.shape)
print(vf2022.shape)

(2516688, 43)
(2749830, 43)
(2936524, 43)
(3383622, 43)
(3335292, 43)
(3625903, 43)
(3514698, 43)
(4649209, 43)
(3803885, 43)


In [5]:
vf = pd.concat([vf2022, vf2021, vf2020, vf2019, vf2018, vf2017, vf2016, vf2015, vf2014], ignore_index=True)


In [6]:
print(vf.columns)

Index(['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain', 'Code service CH'],
      dtype='object')


In [8]:
print(vf["Nature mutation"].value_counts(dropna=False))
print(vf["1er lot"].value_counts(dropna=False))

Nature mutation
Vente                                 27775517
Vente en l'état futur d'achèvement     2085142
Echange                                 405566
Adjudication                            111197
Vente terrain à bâtir                    96519
Expropriation                            41710
Name: count, dtype: int64
1er lot
NaN      21035867
1          326170
2          268312
3          210252
10         179842
           ...   
33K             1
20805           1
7372            1
7644            1
7700            1
Name: count, Length: 22126, dtype: int64


In [9]:
vf_clean = vf[['Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'Type de voie', 'Voie', 'Code postal',
       'Commune', 'Code commune', 
       'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot',
       'Surface Carrez du 3eme lot',
       'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot',
       'Type local','Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain']]

In [12]:
vf_clean[vf_clean["Type local"] == "Appartement"].sample(10)

,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Voie,Code postal,Commune,Code commune,Surface Carrez du 1er lot,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
22385966,29/02/2016,Vente,"145000,00",5250.0,NaN,LE VILLAGE,4150.0,BANON,18,NaN,NaN,NaN,NaN,NaN,Appartement,32.0,2.0,S,NaN,130.0
22258238,06/04/2017,Vente,"170000,00",77.0,RUE,TAITBOUT,75009.0,PARIS 09,109,"19,52",NaN,NaN,NaN,NaN,Appartement,25.0,1.0,NaN,NaN,NaN
5342718,28/10/2021,Vente,"75000,00",25.0,RUE,JEAN JAURES,33230.0,COUTRAS,138,"39,30",NaN,NaN,NaN,NaN,Appartement,40.0,2.0,NaN,NaN,NaN
18650803,26/04/2018,Vente,"684000,00",124.0,BD,DE LA REPUBLIQUE,92210.0,SAINT-CLOUD,64,"132,53",NaN,NaN,NaN,NaN,Appartement,130.0,6.0,NaN,NaN,NaN
14727776,01/10/2019,Vente,"117000,00",1.0,BD,ETIENNE HARDY,77610.0,FONTENAY-TRESIGNY,192,"34,79",NaN,NaN,NaN,NaN,Appartement,51.0,2.0,NaN,NaN,NaN
184050,24/05/2022,Vente,"212000,00",102.0,AV,DE GRASSE,6800.0,CAGNES SUR MER,27,NaN,"65,04",NaN,NaN,NaN,Appartement,65.0,3.0,NaN,NaN,NaN
11401105,20/11/2020,Vente,"124000,00",61.0,AV,DE CLARET,83000.0,TOULON,137,"42,07",NaN,NaN,NaN,NaN,Appartement,42.0,2.0,NaN,NaN,NaN
1680133,20/06/2022,Vente,"200020,00",12.0,RUE,MARIE ANNE DU BOCCAGE,44000.0,NANTES,109,"32,35",NaN,NaN,NaN,NaN,Appartement,32.0,2.0,NaN,NaN,NaN
24719641,01/04/2016,Adjudication,"76000,00",5.0,RUE,HENRI FRENAY,83120.0,SAINTE MAXIME,115,NaN,NaN,NaN,NaN,NaN,Appartement,20.0,1.0,NaN,NaN,NaN
27788377,21/09/2015,Adjudication,"290000,00",35.0,RUE,FERNAND PELLOUTIER,92100.0,BOULOGNE-BILLANCOURT,12,NaN,NaN,NaN,NaN,NaN,Appartement,60.0,3.0,NaN,NaN,NaN


In [ ]:
doublons4 = vf_clean.duplicated().sum()
total_obs4 = len(vf_clean)
ratio_doublons4 = doublons4/total_obs4
print(ratio_doublons4)

In [19]:
outpath2 = 'Valeursfoncières/vf.parquet'

s3.from_pandas_to_parquet_store_in_s3(vf_clean, outpath2)

Fichier Parquet écrit avec succès dans Valeursfoncières/vf.parquet


### Normalisation des adresses

In [ ]:
# Convertir explicitement toutes les colonnes en chaînes
#vf['Adresse'] = vf['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
 #               vf['Type de voie'].fillna('').astype(str) + " " + \
 #               vf['Voie'].fillna('').astype(str) + ", " + \
 #               vf['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
 #               vf['Commune'].fillna('').astype(str)
#print(vf.describe)

In [ ]:
#vf['Adresse'] = vf['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides
#print(vf2022[['No voie', 'Type de voie', 'Voie', 'Code postal', 'Commune', 'Adresse']].head())

In [ ]:
#vf2022['Adresse_Normalisee'] = vf2022['Adresse'].apply(s3.normalize_address)

#print(vf2022[['Adresse', 'Adresse_Normalisee']].head())

TypeError: s3_connection.normalize_address() takes 1 positional argument but 2 were given